In [15]:
from pymongo import MongoClient
from dotenv import dotenv_values
import weaviate
from tqdm import tqdm
import torch
import numpy as np
env_vars = dotenv_values('.env')

In [2]:
weaviate_client = weaviate.Client(
    url = env_vars['WEAVIATE_API_URL'],
    auth_client_secret=weaviate.AuthApiKey(api_key=env_vars['WEAVIATE_API_KEY']),
)

In [3]:
mongo_client = MongoClient('mongodb://{user}:{password}@{host}:{port}/'.format(user=env_vars['MONGO_USER'],
                                                                               password=env_vars['MONGO_PASSWORD'],
                                                                               host=env_vars['MONGO_HOST'],
                                                                               port=env_vars['MONGO_PORT']))

In [4]:
db = mongo_client[env_vars['MONGO_DB']]

In [5]:
collection_name = 'vendor_embeddings'

collection = db[collection_name]

In [6]:
class_obj = {
    "class": "Vendor",
    "vectorizer": "none",  # If set to "none" you must always provide vectors yourself. Could be any other "text2vec-*" also.
}

weaviate_client.schema.create_class(class_obj)

UnexpectedStatusCodeException: Create class! Unexpected status code: 422, with response body: {'error': [{'message': 'class name "Vendor" already exists'}]}.

In [16]:
with weaviate_client.batch(
    batch_size=10
) as batch:
    for document in collection.find():
        name = document['name']
        embedding = document['search']

        properties = {
            "name": name,
        }
        numpy_array = np.array(embedding)

        tensor = torch.from_numpy(numpy_array)

        weaviate_client.batch.add_data_object(
            properties,
            "Vendor",
            vector=tensor
        )

UnexpectedStatusCodeException: Create objects in batch! Unexpected status code: 400, with response body: {'code': 400, 'message': 'parsing body body from "" failed, because json: cannot unmarshal array into Go struct field Object.objects.vector of type float32'}.

In [11]:
for document in collection.find():
    name = document['name']
    embedding = document['search']

    properties = {
        "name": name,
    }
    print(properties)
    #print(embedding)


{'name': 'Aptean'}
{'name': 'Ashleys Pallets'}
{'name': 'Derek Greer'}
{'name': 'John A Bosco'}
{'name': 'Meniscus Group'}
{'name': 'Nexeo'}
{'name': 'Premier Employee Solutions'}
{'name': 'Ready Alliance Protection Group Llc'}
{'name': '4Patriots'}
{'name': 'Alta Staffing  Llc'}
{'name': 'Arrive Logistics'}
{'name': 'Barcode Labeling Systems'}
{'name': 'California Packaging'}
{'name': 'Garrett & Associates'}
{'name': 'Hygiena Llc'}
{'name': 'Interstate Billing Service, Inc.'}
{'name': 'J Rettenmaier Usa Lp'}
{'name': 'Jesse Deramos'}
{'name': 'Narkiss Tapia'}
{'name': 'Nle'}
{'name': 'Oregon Potato Co.'}
{'name': 'Ppc Flexible Packaging'}
{'name': 'Presto Products Company'}
{'name': 'Schnitzer'}
{'name': 'Sensory Effects'}
{'name': 'Sonstegard Foods Co.'}
{'name': 'Sunbelt Rentals Inc'}
{'name': 'Topchance, Inc'}
{'name': 'Tricorbraun Industrial'}
{'name': 'Bakemark'}
{'name': 'Brady Industries, Llc'}
{'name': 'Chano And Sons Inc'}
{'name': 'Gary Bringhurst'}
{'name': 'Grainger'}
{'na

{'name': 'Cristian Salgado'}
{'name': 'Daren Gappmayer\n'}
{'name': 'Diego Barrios Amaya\n'}
{'name': 'Diego Moerira\n'}
{'name': 'Ethan Augason\n'}
{'name': 'Harbor Bay Group, Inc Dba Mr Shrinkwrap'}
{'name': 'Hector Suarez\n'}
{'name': 'Irais Alonso'}
{'name': 'Isaac Williams\n'}
{'name': 'Jacob Overson\n'}
{'name': 'James Dunn'}
{'name': 'Jefferson Mendoza\n'}
{'name': 'Jeffery Smith'}
{'name': 'Jesus Alarcon\n'}
{'name': 'Jill Yonker'}
{'name': 'Juana Lopez-Pedrego'}
{'name': 'Kurt Paxman'}
{'name': 'Larry Gomez\n'}
{'name': 'Lisbeth Barreto Guedez\n'}
{'name': 'Luis Socarras Perez'}
{'name': 'Mark Kamp'}
{'name': 'Maximiliano Larrucea\n'}
{'name': 'Melissa Cabello'}
{'name': 'Michael Gourdine Jr'}
{'name': 'Nicholas Denti\n'}
{'name': 'Paul Kujanpaa\n'}
{'name': 'Ricardo Suarez'}
{'name': 'Romaric Milandou Sita'}
{'name': 'Samual Perry\n'}
{'name': 'Sory Reyes\n'}
{'name': 'Idaho Pacific / Agra West'}
{'name': 'Shawn Rudis'}
{'name': 'Terra Universal'}
{'name': 'Mountain West Cons

In [ ]:
tokens_max_length = tokenizer.model_max_length

def get_embeddings(text):
    tokens = tokenizer.tokenize(text)
    if len(tokens) > tokens_max_length - 2:
        tokens = tokens[:tokens_max_length - 2]
    tokens = ['[CLS]'] + tokens + ['[SEP]']
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_tensor = torch.tensor([input_ids])
    with torch.no_grad():
        embeddings = model(input_tensor)[0]
    return embeddings

get_embeddings("google")

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model =DistilBertModel.from_pretrained('distilbert-base-uncased')